## Question 1 (M)

Propose a supervised learning setting:

- input space $\mathcal{X}$, representing a physical quantity of your choice, with units.
- output space $\mathcal{Y}$, representing a physical quantity of your choice, with units
- a random variable (X, Y) with a joint distribution.
- a loss function l(x, y)

$$
l =
\begin{cases}
\mathcal{X} \times \mathcal{Y} \longrightarrow \mathbb{R_+}
\\
(x,y) \longrightarrow l(x,y)
\end{cases}
$$

Compute the Bayes predictor $f^* : \mathcal{X} \longrightarrow \mathcal{Y}$ and the Bayes risk associated with this
setting. Remark : you have to use a setting different than the settings seen during
the course, in terms of input space X and output space Y. However, you can use any
classical loss function l (square loss, "0-1" loss, etc)

---


## Roulette russe : prédire la survie selon le nombre de balles

- Entrée: $X \in \{1, 2, \dots, 6\}$ : nombre de balles dans le barillet (6 chambres)
- Sortie: $Y \in \{0, 1\}$ : $Y = 1$ si le joueur **survit**, $Y = 0$ sinon

On suppose que:

- X suit une loi uniforme ${U}(1, 6)$
- Le tir est aléatoire : chaque chambre a la même probabilité d'être tirée

La survie dépend du nombre de balles:
  

$$
\mathbb{P}(Y = 1 \mid X = x) = 1 - \frac{x}{6}
$$

Cela signifie que, conditionnellement à $ X = x $, la variable aléatoire $Y$ suit une loi de Bernoulli :

$$
Y \mid X = x \sim \mathrm{Bernoulli}\left(1 - \frac{x}{6} \right)
$$



On utilisera la square loss, donc:


### Prédicteur de Bayes

Dans le cas de la square loss, le prédicteur de Bayes est l'espérance conditionnelle :

$$ f^*(x) = \mathbb{E}[Y \mid X = x] = \mathbb{P}(Y = 1 \mid X = x) = 1 - \frac{x}{6} $$


### Risque de Bayes


\begin{align*}
R(f^*) &= \mathbb{E}_{(X,Y) \sim \rho}[\ell(Y, f^*(X))] \\
&= \mathbb{E}_X \big[ \mathbb{E}_Y[(Y - f^*(X))^2 \mid X] \big] \\
&= \mathbb{E}_X \big[ \operatorname{Var}(Y \mid X) \big] \\
&= \sum_{x=1}^6 \operatorname{Var}(Y \mid X=x) \cdot \mathbb{P}(X = x) \\
&= \sum_{x=1}^6 \left(1 - \frac{x}{6}\right) \frac{x}{6} \cdot \frac{1}{6} \text{ comme Y∣X suit une loi de Bernoulli}\\
&= \frac{1}{6} \sum_{x=1}^6 \left(1 - \frac{x}{6}\right) \frac{x}{6} = \frac{35}{216} \approx 0.162
\end{align*}



## Question 2 (C)

Propose an estimator  $\tilde{f} : \mathcal{X} \rightarrow \mathcal{Y}$, different than the Bayes estimator
and run a simulation that gives a statistical approximation of its generalization
error (risque réel) by computing its empirical risk on a test set. Perform the same
simulation also for $f^*$, and verify that the generalization error is smaller for $f^*$ than
for $\tilde{f}$, and that your the test error for $f^*$ is close to the Bayes risk (this should be the
case if the number of samples is sufficiently large).

---

In [2]:
import numpy as np

n_test = 100000
X_values = np.arange(1, 7)

# Prédicteur de Bayes
def f_star(X):
    return 1 - X / 6

# Estimateur constant 
def f_tilde(X):
    p_mean = np.mean(1 - X_values / 6) # Probabilité moyenne
    return np.full_like(X, p_mean, dtype=float)

def loss(y, y_pred):
    return (y - y_pred)**2

def empirical_risk(f, X_test, Y_test):
    preds = f(X_test)
    return np.mean(loss(Y_test, preds))

X_test = np.random.choice(X_values, size=n_test, replace=True)
Y_test = np.random.binomial(1, 1 - X_test / 6)

risk_f_star = empirical_risk(f_star, X_test, Y_test)
risk_f_tilde = empirical_risk(f_tilde, X_test, Y_test)

# Risque de Bayes théorique
bayes_risk = np.mean((1 - X_values / 6) * (X_values / 6))

print(f"Risque empirique de f* (prédicteur de Bayes) : {risk_f_star:.4f}")
print(f"Risque empirique de f~ (estimateur constant) : {risk_f_tilde:.4f}")
print(f"Risque de Bayes théorique : {bayes_risk:.4f}")


Risque empirique de f* (prédicteur de Bayes) : 0.1618
Risque empirique de f~ (estimateur constant) : 0.2430
Risque de Bayes théorique : 0.1620


Le risque empirique de  $f^*$  est très proche du risque de Bayes théorique 0.1620, ce qui montre que notre simulation est cohérente et bien calibrée.

L’estimateur constant $\tilde{f}$, qui prédit toujours la probabilité moyenne de survie  0.583, obtient un risque empirique plus élevé.

Cela confirme que le prédicteur de Bayes s’adapte mieux aux entrées et minimise effectivement l’erreur quadratique.
